Тензоры
==========================

Тензоры — это специализированная структура данных, очень похожая на массивы и матрицы. В PyTorch мы используем тензоры для кодирования входных и выходных данных модели, а также параметров модели.Тензоры аналогичны `ndarrays` из Numpy, за исключением того, что тензоры могут работать на GPU. На самом деле, тензоры и массивы NumPy часто могут совместно использовать одну и ту же базовую память, что избавляет от необходимости копировать данные (см. bridge-to-np-label). 

Тензоры также оптимизированы для автоматического дифференцирования (узнаем об этом больше в ноутбуке ***06_autogradqs_tutorial***).

In [2]:
import torch
import numpy as np

Инициализация:

In [3]:
data = [[1, 2],[3, 4]]
x_data = torch.tensor(data)

**Из NumPy array**

Тензоры можно создавать из массивов NumPy (и наоборот — см. `bridge-to-np-label`).

In [4]:
np_array = np.array(data)
x_np = torch.from_numpy(np_array)
x_np

tensor([[1, 2],
        [3, 4]], dtype=torch.int32)

**Из другого тензора:**

Новый тензор сохраняет свойства (форма, тип данных) тензора-аргумента, если только он явно не переопределен.

In [5]:
x_ones = torch.ones_like(x_data)
print(f"Ones Tensor: \n {x_ones} \n")

x_rand = torch.rand_like(x_data, dtype=torch.float)
print(f"Random Tensor: \n {x_rand} \n")

Ones Tensor: 
 tensor([[1, 1],
        [1, 1]]) 

Random Tensor: 
 tensor([[0.4625, 0.2511],
        [0.1239, 0.7779]]) 



**С помощью случайных или константных значений:**

In [6]:
shape = (2,3,)
rand_tensor = torch.rand(shape)
ones_tensor = torch.ones(shape)
zeros_tensor = torch.zeros(shape)

print(f"Random Tensor: \n {rand_tensor} \n")
print(f"Ones Tensor: \n {ones_tensor} \n")
print(f"Zeros Tensor: \n {zeros_tensor}")

Random Tensor: 
 tensor([[0.3311, 0.9818, 0.2475],
        [0.2801, 0.8710, 0.6229]]) 

Ones Tensor: 
 tensor([[1., 1., 1.],
        [1., 1., 1.]]) 

Zeros Tensor: 
 tensor([[0., 0., 0.],
        [0., 0., 0.]])


--------------




**Атрибуты тензора:**

1. shape - форма
2. datatype - тип данных
3. device - устройство, на котором хранится тензор (GPU, CPU)

In [11]:
tensor = torch.rand(3,4)

print(f"Shape of tensor: {tensor.shape}")
print(f"Datatype of tensor: {tensor.dtype}")
print(f"Device tensor is stored on: {tensor.device}")

Shape of tensor: torch.Size([3, 4])
Datatype of tensor: torch.float32
Device tensor is stored on: cpu


--------------




Операции над тензорами
~~~~~~~~~~~~~~~~~

Подробнее: <https://pytorch.org/docs/stable/torch.html>.

Каждую операцию можно делать на GPU. По-умолчанию тензоры создаются на CPU. Нам нужно явно переместить тензоы в GPU с помощбю метода `.to`. 

Имейте в виду, что копирование больших тензоров между устройствами  (между CPU и GPU) может быть дорогостоящим с точки зрения времени и памяти!



In [12]:
tensor

tensor([[0.5699, 0.9759, 0.3405, 0.2826],
        [0.5517, 0.6816, 0.8749, 0.8324],
        [0.7364, 0.9462, 0.8741, 0.3306]])

In [14]:
# переводим тензор на GPU, если это возможно
if torch.cuda.is_available():
    tensor = tensor.to('cuda')
tensor

tensor([[0.5699, 0.9759, 0.3405, 0.2826],
        [0.5517, 0.6816, 0.8749, 0.8324],
        [0.7364, 0.9462, 0.8741, 0.3306]], device='cuda:0')

Попробуйте некоторые операции из списка. Если вы знакомы с NumPy API, вам не составит труда использовать Tensor API.

**Стандартное индексирование и слайсинг в стиле numpy:**

In [16]:
tensor = torch.ones(4, 4)
print(f"First row: {tensor[0]}")
print(f"First column: {tensor[:, 0]}")
print(f"Last column: {tensor[..., -1]}")
tensor[:,1] = 0
print(tensor)

First row: tensor([1., 1., 1., 1.])
First column: tensor([1., 1., 1., 1.])
Last column: tensor([1., 1., 1., 1.])
tensor([[1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.]])


Вы можете использовать `torch.cat`для объединения последовательности тензоров по заданному измерению. 

In [20]:
t1 = torch.cat([tensor, tensor, tensor], dim=1)
t1

tensor([[1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1.],
        [1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1.],
        [1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1.],
        [1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1.]])

In [19]:
t2 = torch.cat([tensor, tensor])
t2

tensor([[1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.]])

**Арифметические операции**



In [26]:
# Матричное умножение. y1, y2, y3 будут иметь одинаковое значение.
y1 = tensor @ tensor.T
y2 = tensor.matmul(tensor.T)
y1, y2

(tensor([[3., 3., 3., 3.],
         [3., 3., 3., 3.],
         [3., 3., 3., 3.],
         [3., 3., 3., 3.]]),
 tensor([[3., 3., 3., 3.],
         [3., 3., 3., 3.],
         [3., 3., 3., 3.],
         [3., 3., 3., 3.]]))

In [27]:
y3 =torch.rand_like(y1)
torch.matmul(tensor, tensor.T, out=y3)
y3

tensor([[3., 3., 3., 3.],
        [3., 3., 3., 3.],
        [3., 3., 3., 3.],
        [3., 3., 3., 3.]])

In [28]:
# Поэлементное перемножение
z1 = tensor * tensor
z2 = tensor.mul(tensor)
z3 = torch.rand_like(tensor)
torch.mul(tensor, tensor, out=z3)
z1, z2, z3

(tensor([[1., 0., 1., 1.],
         [1., 0., 1., 1.],
         [1., 0., 1., 1.],
         [1., 0., 1., 1.]]),
 tensor([[1., 0., 1., 1.],
         [1., 0., 1., 1.],
         [1., 0., 1., 1.],
         [1., 0., 1., 1.]]),
 tensor([[1., 0., 1., 1.],
         [1., 0., 1., 1.],
         [1., 0., 1., 1.],
         [1., 0., 1., 1.]]))

**Одноэлементный тензор** 

Если у вас есть одноэлементный тензор, вы можете преобразовать его в числовое значение Python, используя ``item()``:



In [29]:
agg = tensor.sum()
agg

tensor(12.)

In [30]:
agg_item = agg.item()
print(agg_item, type(agg_item))

12.0 <class 'float'>


**In-place операции**

Операции, которые сохраняют результат в операнд, вызываются in-place. Такие операции обозначаются суффиксом `_`. Например, `x.copy_(y)`, `x.t_()` заменит `x`.

In-place операция — это операция, которая напрямую **изменяет содержимое данного тензора без создания копии**. In-place операция в Pytorch всегда имеют постфикс `_`, например `.add_()` или `.scatter_()`. Операции Python, такие как `+=` или `*=`, также являются In-place операциями.

In [31]:
print(f"{tensor} \n")
tensor.add_(5)
print(tensor)

tensor([[1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.]]) 

tensor([[6., 5., 6., 6.],
        [6., 5., 6., 6.],
        [6., 5., 6., 6.],
        [6., 5., 6., 6.]])


<div class="alert alert-info"><h4>Примечание</h4><p>In-place операции экономят немного памяти, но могут быть проблематичными при вычислении производных из-за немедленной потери истории. Поэтому их использование не рекомендуется!</p></div>

--------------




Bridge с NumPy

~~~~~~~~~~
Тензоры, хранящиеся на CPU, и NumPy массивы могут совместно использовать свои базовые области памяти, и изменение одного из них изменит другое.


Из Тензора в NumPy array

In [36]:
t = torch.ones(5)
print(f"t: {t}")
n = t.numpy()
print(f"n: {n}")

t: tensor([1., 1., 1., 1., 1.])
n: [1. 1. 1. 1. 1.]


Изменение Тензора влияет на NumPy array.

In [37]:
t.add_(1)
print(f"t: {t}")
print(f"n: {n}")

t: tensor([2., 2., 2., 2., 2.])
n: [2. 2. 2. 2. 2.]


Из NumPy array в Тензор

In [38]:
n = np.ones(5)
t = torch.from_numpy(n)

Изменение NumPy array влияет на Тензор.

In [39]:
np.add(n, 1, out=n)
print(f"t: {t}")
print(f"n: {n}")

t: tensor([2., 2., 2., 2., 2.], dtype=torch.float64)
n: [2. 2. 2. 2. 2.]


---